<a href="https://colab.research.google.com/github/ankitgurjarr/summer-training/blob/main/day10/Assignment/bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=pd.read_json('/content/day10.json', lines = True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [2]:
###Drop Nan Values
df=df.dropna()


0    14985
1    13634
Name: is_sarcastic, dtype: int64

In [7]:

## Get the Independent Features
X=df.drop(['article_link','is_sarcastic'],axis=1)

In [8]:

## Get the Dependent features
y=df['is_sarcastic']
y.value_counts()

0    14985
1    13634
Name: is_sarcastic, dtype: int64

In [4]:
X.shape

(28619, 1)

In [5]:

y.shape

(28619,)

In [6]:

import tensorflow as tf
tf.__version__

'2.8.2'

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [10]:
### Vocabulary size
voc_size=5000

In [11]:
messages=X.copy()
messages['headline'][1]

'dem rep. totally nails why congress is falling short on gender, racial equality'

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['headline'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
23619
23620
23621
23622
23623
23624
23625
23626
23627
23628
23629
23630
23631
23632
23633
23634
23635
23636
23637
23638
23639
23640
23641
23642
23643
23644
23645
23646
23647
23648
23649
23650
23651
23652
23653
23654
23655
23656
23657
23658
23659
23660
23661
23662
23663
23664
23665
23666
23667
23668
23669
23670
23671
23672
23673
23674
23675
23676
23677
23678
23679
23680
23681
23682
23683
23684
23685
23686
23687
23688
23689
23690
23691
23692
23693
23694
23695
23696
23697
23698
23699
23700
23701
23702
23703
23704
23705
23706
23707
23708
23709
23710
23711
23712
23713
23714
23715
23716
23717
23718
23719
23720
23721
23722
23723
23724
23725
23726
23727
23728
23729
23730
23731
23732
23733
23734
23735
23736
23737
23738
23739
23740
23741
23742
23743
23744
23745
23746
23747
23748
23749
23750
23751
23752
23753
23754
23755
23756
23757
23758
23759
23760
23761
23762
23763
23764
23765
23766
23767
23768
23769
23770
23771
23772
23773
23774
23775
23776
2

In [17]:
corpus

['thirtysometh scientist unveil doomsday clock hair loss',
 'dem rep total nail congress fall short gender racial equal',
 'eat veggi delici differ recip',
 'inclement weather prevent liar get work',
 'mother come pretti close use word stream correctli',
 'white inherit',
 'way file tax less stress',
 'richard branson global warm donat nearli much cost fail balloon trip',
 'shadow govern get larg meet marriott confer room b',
 'lot parent know scenario',
 'lesbian consid father indiana amaz one',
 'amanda peet told daughter sex special hug',
 'know regard current treatment ebola',
 'chri christi suggest hillari clinton blame boko haram kidnap hundr schoolgirl',
 'ford develop new suv run pure gasolin',
 'uber ceo travi kalanick step trump econom advisori council',
 'area boy enter jump touch top doorway phase',
 'area man travel gurney',
 'leav person disabl behind',
 'lin manuel miranda would like remind put phone away',
 'journalist kill target intern press rise',
 'guard video game 

In [16]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr


[[2667, 316, 4566, 3976, 1350, 3452, 2205],
 [1099, 833, 52, 1482, 4531, 4811, 3103, 3892, 3996, 1392],
 [43, 1609, 3931, 3768, 3913],
 [2485, 422, 1804, 2091, 58, 2895],
 [539, 1924, 2389, 712, 2909, 1721, 2586, 2353],
 [4457, 2705],
 [2986, 4280, 2386, 1897, 4425],
 [1697, 864, 2566, 4261, 4496, 357, 197, 3064, 3192, 2068, 1875],
 [4244, 3697, 58, 2495, 1226, 2014, 1513, 1046, 4306],
 [1483, 1024, 1409, 4185],
 [3673, 2802, 1946, 1673, 4288, 2330],
 [1602, 898, 2019, 1755, 2723, 4052, 3188],
 [1409, 769, 3032, 1835, 1517],
 [1183, 3588, 2338, 3154, 2609, 2446, 3632, 883, 1176, 1358, 404],
 [4217, 2883, 2200, 3741, 1226, 3189, 2977],
 [48, 2542, 1431, 942, 717, 4982, 4817, 1288, 4210],
 [4391, 444, 1642, 926, 405, 1439, 3913, 1354],
 [4391, 2553, 4509, 4513],
 [4734, 2914, 2715, 4403],
 [560, 2902, 3651, 3004, 326, 1542, 2703, 455, 2869],
 [4296, 4012, 4671, 1793, 4534, 3895],
 [896, 2956, 190, 523, 4761, 3535, 2816, 2749, 676],
 [4725],
 [2678, 3049, 4210, 4786, 3503],
 [252, 2609, 3

In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)


[[   0    0    0 ... 1350 3452 2205]
 [   0    0    0 ... 3892 3996 1392]
 [   0    0    0 ... 3931 3768 3913]
 ...
 [   0    0    0 ... 2759 2897 2211]
 [   0    0    0 ...  988 4279 4372]
 [   0    0    0 ...  412 4376   51]]


In [20]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 2667,  316, 4566, 3976, 1350, 3452, 2205], dtype=int32)

In [21]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs),y.shape

(28619, (28619,))

In [24]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)


In [25]:
X_final.shape,y_final.shape

((28619, 20), (28619,))

In [26]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [27]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
300/300 [==============================] - 13s 34ms/step - loss: 0.5613 - accuracy: 0.7000 - val_loss: 0.4965 - val_accuracy: 0.7503
Epoch 2/10
300/300 [==============================] - 13s 42ms/step - loss: 0.4272 - accuracy: 0.8049 - val_loss: 0.4985 - val_accuracy: 0.7554
Epoch 3/10
300/300 [==============================] - 15s 49ms/step - loss: 0.3699 - accuracy: 0.8385 - val_loss: 0.5198 - val_accuracy: 0.7466
Epoch 4/10
300/300 [==============================] - 11s 37ms/step - loss: 0.3236 - accuracy: 0.8628 - val_loss: 0.5531 - val_accuracy: 0.7508
Epoch 5/10
300/300 [==============================] - 18s 59ms/step - loss: 0.2895 - accuracy: 0.8803 - val_loss: 0.5755 - val_accuracy: 0.7455
Epoch 6/10
300/300 [==============================] - 10s 33ms/step - loss: 0.2605 - accuracy: 0.8975 - val_loss: 0.6522 - val_accuracy: 0.7401
Epoch 7/10
300/300 [==============================] - 10s 35ms/step - loss: 0.2304 - accuracy: 0.9095 - val_loss: 0.6798 - val_accuracy:

In [29]:
y_pred1 = (model.predict(X_test) > 0.5).astype("int32")


In [30]:
from sklearn.metrics import confusion_matrix

In [31]:

confusion_matrix(y_test,y_pred1)

array([[1756, 3160],
       [1753, 2776]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.4798305982001059

In [33]:

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.50      0.36      0.42      4916
           1       0.47      0.61      0.53      4529

    accuracy                           0.48      9445
   macro avg       0.48      0.49      0.47      9445
weighted avg       0.48      0.48      0.47      9445

